In [1]:
import bs4,time,pyperclip
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys



driver_loc = "./chromedriver.exe"
options = webdriver.ChromeOptions()
options.add_argument("window-size = 1920x1080")
options.add_argument('headless')                           
options.add_argument('disable-gpu')
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")

driver = webdriver.Chrome(driver_loc, options = options)
driver.maximize_window()
url = 'https://www.youtube.com/'
driver.get(url)
time.sleep(1.5)

# 검색어 
inputSearch = '스마트밴드'

# 변수로 지정 시 해당 요소를 찾지 못하면 오류가 발생
tag_search = driver.find_element_by_name('search_query')
search_btn = driver.find_element_by_id('search-icon-legacy')

tag_search.click()
pyperclip.copy(inputSearch)
tag_search.send_keys(Keys.CONTROL, 'v')


time.sleep(0.5)

search_btn.click()
time.sleep(2)
driver.switch_to_default_content # 처음 iframe을 걷어내고 찾아야함


filter_btn = driver.find_element_by_link_text('필터')


filter_btn.click()
time.sleep(1)

# 변수로 넣어주는 경우에도 해당 요소를 찾지 못하면 계속 오류가 발생
thisYear_btn = driver.find_element_by_link_text('올해')
thisYear_btn.click()
time.sleep(1)

filter_btn.click()
time.sleep(1)

views_btn = driver.find_element_by_link_text('조회수')
views_btn.click()
time.sleep(2)

result = []
for i in range(0,5):
    column = []
    searchResult = '//*[@id="contents"]/ytd-video-renderer[{}]'.format(i+1)
    driver.find_element_by_xpath(searchResult).click()
    time.sleep(2)
    try:
        driver.find_element_by_tag_name('body').send_keys(Keys.SPACE)
    except:
        print("드라이버동작 불안정")
        pass
    time.sleep(3)
       
    data = driver.page_source
    bs = bs4.BeautifulSoup(data, 'html.parser')
    

    divInfoTag = bs.find(name = 'div',
                attrs = {'id': 'info-contents'})

    divMetaTag = bs.find(name = 'div',
                attrs = {'id': 'meta-contents'})

       
#     영상 제목
    vidTitle = divInfoTag.find(name = 'h1',
                   attrs = {'class': 'title style-scope ytd-video-primary-info-renderer'}).text
    
#     영상 조회수
    vidCount = divInfoTag.find(name = 'span',
                   attrs = {'class': 'view-count style-scope yt-view-count-renderer'}).text
#     영상 날짜
    vidDate = divInfoTag.find(name = 'div',
                       attrs = {'id': 'date'}).text.replace("•","").replace(" ","")
                   

    
#     추천수
    vidRecommand = divInfoTag.find(name = 'paper-tooltip',
                       attrs = {'class': 'style-scope ytd-sentiment-bar-renderer'}).text.replace("\n","").replace(" ","")
                       
#     채널명
    vidChannel = divMetaTag.find(name = 'paper-tooltip' ,
                       attrs = {'class': 'style-scope ytd-channel-name'}).text.replace("\n","").replace(" ","")
    
    column = [vidChannel, vidTitle, vidCount, vidDate, vidRecommand]
    result.append(column)
    
    num_page_down = 20
    while num_page_down:
        driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        num_page_down -= 1
        
    driver.switch_to_default_content 
        
    
    data = driver.page_source
    bs = bs4.BeautifulSoup(data, 'html.parser')
    #     댓글
        
    allCommentTags = bs.findAll(name = 'ytd-comment-renderer',
                               attrs = {'class': 'style-scope ytd-comment-thread-renderer'})

    rows = []
    print(len(allCommentTags))
    for j in range(0, len(allCommentTags)):
        columns1 = []
        try:
            comment = allCommentTags[j].find(name = 'div',
                                        attrs = {'id':'content',
                                                 'class':'style-scope ytd-expander'}).text.replace('\n',' ')
        except:
            comment = '1'
    
        
        colunms1 = [1,comment,3]
        rows.append(colunms1)
        
    
    result1 = pd.DataFrame(data = rows, columns=["id","댓글","공감수"])
    print(result1)
    result1.to_csv('./{}{}{}.csv'.format(i+1,'. ',vidChannel +"댓글"), index = False )
    
    driver.back()
    time.sleep(2)

driver.close()
df_result = pd.DataFrame(data = result, columns = ['채널명','제목','조회수','날짜','추천/비추천'])
df_result.to_csv('./{}.csv'.format(inputSearch), index = False, encoding = 'ms949')


